# Data preparation
Prepare data and files for markup for project on the aspect-based summarization of reviews.

## Imports

In [ ]:
from copy import deepcopy
import re
import pandas as pd

## Data

In [ ]:
with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_hrefs.txt', 'r', encoding='utf-8') as h:
    hrefs = h.read().split('\n')[:-1]

with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_names.txt', 'r', encoding='utf-8') as n:
    names = n.read().split('\n')[:-1]

with open('/content/drive/MyDrive/Summarization/laptop data/data/laptop_reviews.txt', 'r', encoding='utf-8') as r:
    reviews = r.read().split('\n')[:-1]

In [ ]:
print(len(hrefs))
print(len(names))
print(len(reviews))

1299
1299
12990


In [ ]:
print(reviews[30])

Плюсы: -Хорошая видеокарта -Памяти 4гб, что очень хорошо -HDMI м eSATA Минусы: -Батарея слабая, фильм еле хватило посмотреть -Греется сильно -Жесткий диск маловат Отзыв: Купили в апреле прошлого года, работает отлично. Винчетре хотелось бы побольше, но есть внешний на 2тб, так что терпимо. Греется довольно сильно, думал взять подставку с охлаждением, но не нашел подходящей в городе. Решил проблему путем подкладывания широкой книги средней толщины, играть даже удобнее стало и экран стал чуть выше располагаться. Долго разбирался как настроить цвет на экране, все равно не получилось желаемого результата, цвета не очень естественные но не сильно бросается в глаза.  Минусы не так значительны, достоинств у него больше чем недостатков. Количество звёзд на отзыве: 4


## Database initialization (base)

It will be а database with both reviews and their markup. The guidelines were provided by the author of this notebook.
* Aspect extraction and sentiment annotation markup: [here]()
* Summarization markup: [here]()

In [ ]:
# products: id short_name href
products_df = pd.DataFrame(
    {
        'id': range(len(names)),
        'short_name': names,
        'href': hrefs
    }
)

products_df.to_csv('/content/drive/MyDrive/Summarization/laptop data/datasets/products.tsv', sep='\t', index=False)

In [ ]:
PATTERN_PLUS = re.compile('\s*Плюсы:\s*')
PATTERN_MINUS = re.compile('\s*Минусы:\s*')
PATTERN_REVIEW = re.compile('\s*Отзыв:\s*')
PATTERN_ADVANTAGES = re.compile('\s*Достоинства:\s*')
PATTERN_DISADVANTAGES = re.compile('\s*Недостатки:\s*')
PATTERN_COMMENT = re.compile('\s*Комментарий:\s*')
PATTERN_STARS = re.compile('\s*Количество звёзд на отзыве:\s*')

PATTERNS = [[PATTERN_PLUS, PATTERN_ADVANTAGES], [PATTERN_MINUS, PATTERN_DISADVANTAGES], [PATTERN_REVIEW, PATTERN_COMMENT]]

In [ ]:
match_1 = 1
match_2 = None

bool(match_1 and match_2)

False

In [ ]:
def get_parts(review: str) -> dict:
    '''
    Get parts of a review.
    '''

    parts = ['', '', '', '']  # pos, neg, rev, stars

    # find relevant parts by regex

    matches = [None, None, None]  # plus, minus, comment

    for idx, (pattern_1, pattern_2) in enumerate(PATTERNS):
        match_1 = re.search(pattern_1, review)
        match_2 = re.search(pattern_2, review)
        if match_1 and match_2:
            if match_1.span()[0] < match_2.span()[0]:
                matches[idx] = pattern_1
            else:
                matches[idx] = pattern_2
        elif match_1:
            matches[idx] = pattern_1
        elif match_2:
            matches[idx] = pattern_2

    # print(matches)

    # split sentiment parts (plus and minus) by patterns

    matches_idx = [idx for idx, pattern in enumerate(matches) if pattern]

    # print(matches_idx)

    try:

        if len(matches_idx) == 3:
            _, part_1 = re.split(matches[0], review, maxsplit=1)
            part_1, part_2 = re.split(matches[1], part_1, maxsplit=1)
            part_2, part_3 = re.split(matches[2], part_2, maxsplit=1)
            part_3, stars = re.split(PATTERN_STARS, part_3, maxsplit=1)

            parts[0], parts[1], parts[2], parts[3] = part_1, part_2, part_3, stars

        elif len(matches_idx) == 2:
            _, part_1 = re.split(matches[matches_idx[0]], review, maxsplit=1)
            part_1, part_2 = re.split(matches[matches_idx[1]], part_1, maxsplit=1)
            part_2, stars = re.split(PATTERN_STARS, part_2, maxsplit=1)
                
            parts[matches_idx[0]], parts[matches_idx[1]], parts[3] = part_1, part_2, stars

        elif len(matches_idx) == 1:
            _, part_1 = re.split(matches[matches_idx[0]], review, maxsplit=1)
            part_1, stars = re.split(PATTERN_STARS, part_1, maxsplit=1)

            parts[matches_idx[0]], parts[3] = part_1, stars

        elif len(matches_idx) == 0:
            rev, stars = re.split(PATTERN_STARS, review, maxsplit=1)

            parts[2], parts[3] = rev, stars

    except ValueError:

        rev, stars = re.split(PATTERN_STARS, review, maxsplit=1)

        parts[2], parts[3] = rev, stars

    return parts

In [ ]:
# if matches[0]:
#         pos = re.sub(matches[0], '', review)
#         print(pos)
#     if matches[1] and matches[0]:
#         pos, neg = re.split(matches[1], pos, maxsplit=1)  # sometimes there are repeated start like 'Плюсы: Плюсы:'
#         print(pos)
#         print(neg)
#     if matches[1] and not matches[0]:
#         _, neg = re.split(matches[1], pos, maxsplit=1)
#     if matches[2] and matches[1]:
#         print(neg)
#         neg, rev = re.split(matches[2], neg, maxsplit=1)
#         print(rev)
#     _, stars = re.split(PATTERN_STARS, review, maxsplit=1)

In [ ]:
pos = []
neg = []
rev = []
stars = []

print(reviews[3937])
parts = get_parts(reviews[3937])
pos.append(parts[0])
neg.append(parts[1])
rev.append(parts[2])
stars.append(parts[3])

print(pos)
print(neg)
print(rev)
print(stars)

Плюсы:Не выявил  Минусы:Windows Срок использования 2 дня Ноутбук даже не включился, точнее, виндовс не запустился, пишет ошибку и ничего сделать нельзя. Никакого установочного диска или флешки в комплекте нет, так что если произойдёт программный сбой, увы и ах на помойку. К сожалению, мы не проверили ноутбук в магазине, т.к. торопились к себе в город, а мы живем в другом городе где нет М.Видео, и попросту нет возможности доставить или забрать товар. Собираемся возвращать по гарантии, надеюсь, вернут. Количество звёзд на отзыве: 1
['Не выявил']
['Windows Срок использования 2 дня Ноутбук даже не включился, точнее, виндовс не запустился, пишет ошибку и ничего сделать нельзя. Никакого установочного диска или флешки в комплекте нет, так что если произойдёт программный сбой, увы и ах на помойку. К сожалению, мы не проверили ноутбук в магазине, т.к. торопились к себе в город, а мы живем в другом городе где нет М.Видео, и попросту нет возможности доставить или забрать товар. Собираемся возвращ

In [ ]:
hrefs[0]

'/products/117174/noutbuki/apple-macbook-pro-15-with-retina-display-mid-2015-mjlq2-core-i7-2200-mhz1542880x1800160gb256gbdvd-netintel-iris-pro-graphics-5200wi-fibluetoothmacos-x/'

In [ ]:
pos = []
neg = []
rev = []
stars = []

for idx, review in enumerate(reviews):
    try:
        parts = get_parts(review)
        pos.append(parts[0])
        neg.append(parts[1])
        rev.append(parts[2])
        stars.append(parts[3])
    except (ValueError, UnboundLocalError, TypeError):
        print(idx)
        print(review)

In [ ]:
# reviews: id product_id text pluses minuses review stars
reviews_df = pd.DataFrame(
    {
        'id': range(0, len(reviews)),
        'product_id': [i for i in range(len(names)) for x in range(10)],
        'text': reviews,
        'pluses': pos,
        'minuses': neg,
        'review': rev,
        'stars': stars
    }
)

reviews_df.to_csv('/content/drive/MyDrive/Summarization/laptop data/datasets/reviews.tsv', sep='\t', index=False)

In [ ]:
# aspects and sentiment: id text_id category sent_from sent_to sentiment sent sent_term normalized_sent_term term_from term_to term type
aspects_df = pd.DataFrame(
    {
        'id': [],
        'text_id': [],
        'category': [],
        'sent_from': [],
        'sent_to': [],
        'sentiment': [],
        'sent': [],
        'sent_term': [],
        'normalized_sent_term': [],
        'term_from': [],
        'term_to': [],
        'term': [],
        'normalized_term': [],
        'type': []
    }
)

aspects_df.to_csv('/content/drive/MyDrive/Summarization/laptop data/datasets/aspects.tsv', sep='\t', index=False)

In [ ]:
# summarization: id product_id term sentiment
summarization_df = pd.DataFrame(
    {
        'id': [],
        'product_id': [],
        'term': [],
        'sentiment': []
    }
)

summarization_df.to_csv('/content/drive/MyDrive/Summarization/laptop data/datasets/summarization.tsv', sep='\t', index=False)